<h1>Matt's eCFR API Demonstrator</h1>
<p>An intro to Jupyter Notebooks</p>
<p><i>Teach yourself Python using the eCFR API</i></p>

In [57]:
# Globals
# Do my imports - datetime for date operations, pandas for data analysis, aiohttp asyncio requests for HTTPS REST API requests, json for, well, javascript object notation, duh
# Note: the nest_asyncio is used to allow asyncio to run in Jupyter Notebooks without blocking the event loop
import datetime as dt
import pandas as pd
import aiohttp as aio
import asyncio as asy   
import requests as rq
import json as json
import nest_asyncio
nest_asyncio.apply()

# Set my constants
base_url = "https://www.ecfr.gov/api/"
titles_url = base_url + "versioner/v1/titles.json"
agencies_url = base_url + "admin/v1/agencies.json"
title_and_chapter_versions_url = base_url + "versioner/v1/versions/title-"
data_path = "../data/"

In [58]:
# Define my eCFR API wrapper class
class eCFR_API_Wrapper:
    """
    A wrapper for the eCFR API to fetch and process data.
    """

    def __init__(self, base_url):
        self.base_url = base_url

    def fetch_data(endpoint):
        """
        Fetch data from the eCFR API.
        """
        try:
            response = rq.get(f"{endpoint}")
            if response.status_code == 200:
                return response.json()
            else:
                response.raise_for_status()
        except: 
            print(f"Error fetching data from {endpoint}")
            return None
        
    async def fetch_data_asynch(endpoint):    
        """
        Fetch data from the eCFR API asynchronously.
        """
        session = rq.Session()
        async with session.get(endpoint) as response:
            # Return the JSON content of the response using 'response.json()'
            return await response.json()

In [ ]:
class eCFR_HouseKeeping:

    titles_json = None
    agencies_json = None
    slugs = []
    titles_and_chapters = []


    def __init__(self, data_path):
        self.data_path = data_path

    # FETCH DATA METHODS
    def fetch_agencies_data(self, asynch=False):
        """
        Fetch the agencies data from the eCFR API.
        """
        try:           
            if( asynch ): 
                print("Fetching agencies asynchronously from the API...")
                self.agencies_json = eCFR_API_Wrapper.fetch_data_asynch(agencies_url)
                return self.agencies_json
            else:
                # fetch the agencies data from the API
                print("Fetching agencies data from the API...") 
                self.agencies_json = eCFR_API_Wrapper.fetch_data(agencies_url)
                return self.agencies_json
        except Exception as e:
            print(f"Error fetching agencies data: {e}")
            return None    
    
    def fetch_titles_data(self, asynch=False):
        """
        Fetch the titles data from the eCFR API.
        """
        try:
            if( asynch ):
                print("Fetching titles asynchronously from the API...")
                self.titles_json = eCFR_API_Wrapper.fetch_data_asynch(titles_url)
                return self.titles_json
            else:
                # fetch the titles data from the API
                print("Fetching titles data from the API...")   
                self.titles_json = eCFR_API_Wrapper.fetch_data(titles_url)
                return self.titles_json
        except Exception as e:
            print(f"Error fetching titles data: {e}")
            return None 

    def fetch_full_text_by_title_and_chapter(self, asynch=True):
        """
        Fetch the full text data from the eCFR API.
        """
        # Note: This method is not implemented in the original code, so we will just return None
        print("Fetching full text data from the API is not implemented.")
        return None 

    
    # LOAD DATA METHODS    
    def load_agency_data(self):
        """
        Load the agency data from the agencies.json file.
        """
        try:
            with open(self.data_path + 'agencies.json', 'r') as myagenciesfile:
                self.agencies_json = json.load(myagenciesfile)
                return self.agencies_json
        except FileNotFoundError:
            print("No agencies.json found, need to refresh")
            self.refresh_agencies()
            return self.agencies.json

    def load_title_data(self):
        """
        Load the title data from the titles.json file.
        """
        try:
            with open(self.data_path + 'titles.json', 'r') as mytitlesfile:
                self.titles_json = json.load(mytitlesfile)
                return self.titles_json
        except FileNotFoundError:
            print("No agencies.json found, need to refresh")
            self.refresh_titles()
            return self.titles_json
        
    def load_full_text_data_by_title_and_chapter(self, title,chapter):
        """
        Load the title and chapter versions from an individual file.
        """
        try:
            with open(self.data_path + 'versions/title'+str(title)+'chapter'+str(chapter)+"_versions.json", 'r') as mytitlesfile:
                title_and_chapter_versions = json.load(mytitlesfile)
                return title_and_chapter_versions
        except FileNotFoundError:
            print("No title"+title+" chapter"+chapter+" versions found, need to fetch data from API")
            return None 

    # REFRESH DATA METHODS        
    def refresh_agencies(self, asynch=False):
        """
        Fetch the agencies data from the eCFR API.
        """
        # fetch the agencies data from the API
        self.fetch_agencies_data(asynch=asynch)
        # if we retrieved agencies_json, write it to the file
        if self.agencies_json:
            try:
                with open(self.data_path + 'agencies.json', 'w') as myagenciesfile:
                    json.dump(self.agencies_json, myagenciesfile)
                return self.agencies_json
            except IOError as e:
                return (f"Error writing to agencies.json: {e}")
        else:
            return None

    def refresh_titles(self, asynch=False):
        """
        Fetch the titles data from the eCFR API.
        """
        # first, see if the titles.json file exists and is fresh    
        isfresh = False
        try:
            with open(self.data_path + 'titles.json', 'r') as mytitlesfile:
                self.titles_json = json.load(mytitlesfile)
        except FileNotFoundError:
            self.titles_json = None 
        if(self.titles_json is not None):
            # Get the current date and calculate the date 30 days ago
            thirtydaysback = dt.datetime.now() - dt.timedelta(days=30)
            lastbestdate = None
            #attempt to get data from the titles.json file
            if(self.titles_json.get('titles', [])):
                # check the up_to_date_as_of field and populate lastbestdate
                titles_dict = self.titles_json.get('titles', [])
                key_to_find = "up_to_date_as_of"
                up_to_date_values = [t[key_to_find] for t in titles_dict if key_to_find in t]
                lastbestdate = max(list(filter(None, up_to_date_values)))       
            if lastbestdate > str(thirtydaysback):
                print(f" with good data as of {lastbestdate}")
                # we have fresh data, so we can use it as is
                isfresh = True

        if( not isfresh ):
            # not fresh, so fetch the titles data from the API
            self.fetch_titles_data(asynch)
            #we have fresh titles_json, write it to the file
            try:
                with open(self.data_path + 'titles.json', 'w') as mytitlesfile:
                    json.dump(self.titles_json , mytitlesfile)
                return self.titles_json
            except IOError as e:
                return None 

    def refresh_full_text_by_title_and_chapter(self, title, chapter, asynch=False):    
        """
        Refresh the title and chapter version data.
        """
        my_title_url = title_and_chapter_versions_url + title + ".json?chapter=" + chapter 
        # fetch the text for a single title and chapter 
        try:
            if(not asynch):
                title_and_chapter_version_json = eCFR_API_Wrapper.fetch_data(my_title_url)
            else:
                title_and_chapter_version_json = eCFR_API_Wrapper.fetch_data_asynch(my_title_url)
        except Exception as e:
            return (f"Error fetching title version data: {e}")
        # if we have version data, write it to the file
        if title_and_chapter_version_json:
            myfile = "title"+title+"chapter"+chapter+"_versions.json"
            try:
                with open(self.data_path + 'versions/'+myfile, 'w') as myversionfile:
                    json.dump(title_and_chapter_version_json, myversionfile)
                return "Version data available, data fetched and written to "+myfile
            except IOError as e:
                return (f"Error writing to versions.json: {e}")
        else:
            return "Unable to fetch versions data from the API"




<h2>Unit Tests</h2>

In [ ]:
import unittest

class TestECFRHouseKeeping(unittest.TestCase):
    def setUp(self):
        self.ecfr_hk = eCFR_HouseKeeping(data_path)

    def test_fetch_agencies_data(self):
        agencies = self.ecfr_hk.fetch_agencies_data()
        self.assertIsNotNone(agencies)

    async def test_fetch_agencies_data_asynch(self):
        # Use 'asyncio.gather()' to run the tasks concurrently and gather their results
        agencies = await asy.gather(self.ecfr_hk.fetch_agencies_data(asynch=True)  )
        self.assertIsNotNone(agencies) 

    def test_fetch_titles_data(self):
        titles = self.ecfr_hk.fetch_titles_data()
        self.assertIsNotNone(titles)

    async def test_fetch_titles_data_asynch(self):
        # Use 'asyncio.gather()' to run the tasks concurrently and gather their results
        titles = await asy.gather(self.ecfr_hk.fetch_titles_data(asynch=True)  )
        self.assertIsNotNone(titles) 

    def test_load_agency_data(self):
        agencies = self.ecfr_hk.load_agency_data()
        self.assertIsNotNone(agencies)

    def test_load_title_data(self):
        titles = self.ecfr_hk.load_title_data()
        self.assertIsNotNone(titles)    




    def test_refresh_agencies(self):
        result = self.ecfr_hk.refresh_agencies()
        self.assertIsNotNone(result)

    def test_refresh_titles(self):
        result = self.ecfr_hk.refresh_titles()
        self.assertIsNotNone(result) 
    
    def test_refresh_title_and_chapter_versions(self):
        result = self.ecfr_hk.refresh_title_and_chapter_versions("1", "I")
        self.assertIn("Version data available", result)


'''
    def test_asynch_request_test(self):
        result = self.asynch_test.get_all_versions()
        self.assertIsNone(result)   
    

    def test_load_title_and_chapter_versions(self):
        versions = self.ecfr_hk.load_title_and_chapter_versions("1", "I")
        self.assertIsNotNone(versions)

    def test_refresh_all_versions_byslug(self):
        result = self.ecfr_hk.refresh_all_versions_byslug()
        self.assertIn("Versions fetched successfully", result)  
'''

unittest.main(argv=[''], verbosity=2, exit=False)


test_fetch_agencies_data (__main__.TestECFRHouseKeeping.test_fetch_agencies_data) ... ok
test_fetch_titles_data (__main__.TestECFRHouseKeeping.test_fetch_titles_data) ... 

Fetching agencies data from the API...
Fetching titles data from the API...


ok
test_fetch_titles_data_asynch (__main__.TestECFRHouseKeeping.test_fetch_titles_data_asynch) ... /usr/local/python/3.12.1/lib/python3.12/unittest/case.py:589: RuntimeWarning: coroutine 'TestECFRHouseKeeping.test_fetch_titles_data_asynch' was never awaited
  if method() is not None:
/usr/local/python/3.12.1/lib/python3.12/unittest/case.py:692: DeprecationWarning: It is deprecated to return a value that is not None from a test case (<bound method TestECFRHouseKeeping.test_fetch_titles_data_asynch of <__main__.TestECFRHouseKeeping testMethod=test_fetch_titles_data_asynch>>)
  return self.run(*args, **kwds)
ok
test_load_agency_data (__main__.TestECFRHouseKeeping.test_load_agency_data) ... ok
test_load_title_data (__main__.TestECFRHouseKeeping.test_load_title_data) ... ok
test_refresh_agencies (__main__.TestECFRHouseKeeping.test_refresh_agencies) ... ok
test_refresh_title_and_chapter_versions (__main__.TestECFRHouseKeeping.test_refresh_title_and_chapter_versions) ... ok
test_refresh_title

Fetching agencies data from the API...
 with good data as of 2025-07-31


<h2>Put classes and constants above<h2>
<h3>Put implementation below</h3>

In [61]:

# initialize variables
titles_json= {'titles':[]}  #the titles_json dictionary to contain data
statusmsg = "initialized"  #use this to track what's going on 
lastbestdate = "1776-07-04"  #when was the eCFR last updated?
today = dt.date.today() #what day is it?

# test successful initialization
assert len(titles_json)>0
assert len(statusmsg)>0
assert str(today)>lastbestdate

print("Welcome to the MRWeCFR with base_url of "+base_url)

# Psuedocode
# 1. Let's see if we have a list of recent titles (less than 30 days old) in our titles.json file
# 2. If we do, let's see if we have the full and amendments of similar age, else, try and refresh titles.json 
# 3. If we are good with source data, skip to analysis, else, try and refresh the eCFR json details 
# 4. Analysis - let's load some basic semantic checks (wordcount, reading level) and some change trackers (count of changes, frequency over time)
# To do the analysis, we're going to have to download 2 things - the full text (for wordcount and reading level) and the amendments (for change tracking).


#let's check the titles.json file for freshness
# ecfr_housekeeping = eCFR_HouseKeeping(data_path)

# implementation to follow






Welcome to the MRWeCFR with base_url of https://www.ecfr.gov/api/
